In [339]:
#Import libraries
import numpy as np
import pandas as pd
import requests
import xlsxwriter 
import math
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from numpy import hstack

In [341]:
#Read both files in
ExchRates = pd.read_csv('ExchRates.csv', index_col=0)
FX_Daily = pd.read_csv('FX-Daily.csv', index_col=0)

#Convert date column into readable dates and find common dates
ExchRates.index = pd.to_datetime(ExchRates.index, format='%Y%m%d')
FX_Daily.index = pd.to_datetime(FX_Daily.index, format='%Y%m%d')
CommonRows = np.intersect1d(ExchRates.index, FX_Daily.index)

#Keep rows that are common between data sets (dates are identical)
ExchRates = ExchRates.loc[CommonRows]
FX_Daily = FX_Daily.loc[CommonRows]

#Keep columns that are common between date sets (currencies are identical)
CommonColumns = ['CAD', 'EUR', 'NOK', 'SEK', 'CHF', 'GBP', 'AUD', 'JPY', 'NZD']
ExchRates = ExchRates[CommonColumns]
FX_Daily = FX_Daily[CommonColumns]
#ExchRates = ExchRates.sort_index(axis=1)
#FX_Daily = FX_Daily.sort_index(axis=1)

#Shift FX_Daily two steps forward to when it comes avaiable
#Shift ExchRates data 7 steps forward as we need to predict a week in advance
FX_Daily = FX_Daily.shift(2)
ExchRates = ExchRates.shift(-7)

In [342]:
#Trim dates so null values are not included post-shift
after_start_date = FX_Daily.index >= "2007-04-26"
before_end_date = FX_Daily.index <= "2020-12-28"
between_two_dates = after_start_date & before_end_date
ExchRates = ExchRates.loc[between_two_dates]
FX_Daily = FX_Daily.loc[between_two_dates]

In [343]:
dateDF = ExchRates.index

In [344]:
#Create a Dictionary of dataframes with the respective FX_Daily and ExchRates columns for each company zipped together
d = {}
for column in CommonColumns:
    d[column] = pd.concat([FX_Daily[column], ExchRates[column]], axis=1, keys=[column + ' FX_Daily', column + ' ExchRates' ])

In [375]:
cRMSE = []
OneWeekVal = []
OneWeekPriceChange = []

In [455]:
#Choose currency of interest
CurrencyOfInterest = 'NZD'
datasetdf = d[CurrencyOfInterest].astype('float32')
#dataset[CurrencyOfInterest + ' ExchRates'] = np.log(dataset[CurrencyOfInterest + ' ExchRates'])
#dataset = datasetdf.to_numpy()
#dataset = dataset.tolist()
dataFX = datasetdf[CurrencyOfInterest + ' FX_Daily'].to_numpy()
dataER = datasetdf[CurrencyOfInterest + ' ExchRates'].to_numpy()
#dataActual = dataER[-7:]

In [441]:
#Split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
#Define input sequence
in_seq1 = dataFX[:-7]
out_seq = dataER[:-7]
#Convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
#Horizontally stack columns
dataset = hstack((in_seq1, out_seq))
#Choose a number of time steps
n_steps = 1
#Convert into input/output
X, y = split_sequences(dataset, n_steps)
#Print(X.shape, y.shape)
#Summarize the data
#For i in range(len(X)):
    #Print(X[i], y[i])

In [442]:
n_features = X.shape[2]
#Define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
#Fit model
model.fit(X, y, epochs=200, verbose=0)
#Demonstrate prediction
x_input = X[-7:]
x_input = x_input.reshape((7, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[1.3816268]
 [1.3929921]
 [1.3731741]
 [1.3866082]
 [1.394301 ]
 [1.3899266]
 [1.3969738]]


In [454]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(dataActual.reshape(-1,1), yhat)

rmse = math.sqrt(mse)
cRMSE.append(rmse)

In [444]:
#Split a multivariate sequence into samples
def split_sequencesforpricechange(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
#Define input sequence
in_seq1 = dataFX[:-1]
out_seq = dataER[:-1]
#Convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
#Horizontally stack columns
dataset = hstack((in_seq1, out_seq))
#Choose a number of time steps
n_steps = 1
#Convert into input/output
X, y = split_sequencesforpricechange(dataset, n_steps)
#Print(X.shape, y.shape)
#Summarize the data
#For i in range(len(X)):
    #Print(X[i], y[i])

In [445]:
n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = X[-1:]
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[1.3880926]]


In [453]:
yhat = yhat.tolist()
OneWeekVal.append(yhat[0][0])

In [452]:
newP=yhat[0][0]
oldP= dataER[-7:-6][0]
pctchange = ((newP-oldP)/oldP)*100
OneWeekPriceChange.append(pctchange)

In [449]:
RSMEdf = pd.DataFrame({'Currency': ['CAD', 'EUR', 'NOK', 'SEK', 'CHF', 'GBP', 'AUD', 'JPY', 'NZD'], 'T+1 Price':OneWeekVal, 'T+1 Price Change':OneWeekPriceChange,'RMSE': cRMSE})
RSMEdf = RSMEdf.set_index('Currency')
RSMEdf

,T+1 Price,T+1 Price Change,RMSE
Currency,,,
CAD,1.194404,-6.818198,0.102099
EUR,0.780609,-4.372336,0.006269
NOK,7.023805,-18.516394,1.605652
SEK,7.713229,-5.986676,0.393708
CHF,0.988790,11.828758,0.115737
GBP,0.655035,-11.553491,0.077385
AUD,1.226555,-6.697457,0.069543
JPY,102.503571,-1.039227,2.986086
NZD,1.388093,-0.758373,0.015461


In [450]:
RSMEdf.to_csv('G9.csv')